### Import Libraries

In [1]:
"If it does not work use code below while running this notebook in this dir."
# import sys
# import os
# sys.path.append(os.path.abspath(os.getcwd()))

from mytorch import Tensor, Model
from mytorch import activation as active_func
from mytorch import loss as loss_func
from mytorch import optimizer as optim 
from mytorch import layer as nn
from mytorch.util import DataLoader

from mytorch.util import flatten

import matplotlib.pyplot as plt

### Load MNIST

In [ ]:
"""
load data set with given data loader.
you have 10000 train data, 1000 for each number, remember to shuffle training data.
you have 1000 test data, 100 for each number.

loaded data is a list of (img, label)
type of img is Tensor.

TODO: you have to get this list and create batches for training.
you can also apply this changes later in the Training part for convenience.
"""
data_loader = DataLoader(train_addr='MNIST/train', test_addr='MNIST/test')
data_loader.load()
"you can see how data is loaded"
print(data_loader.getTrain()[0][0].shape)
print(data_loader.getTrain()[0][1].shape)

### Define and Create your Model

In [3]:
from mytorch import activation

class MyModel(Model):
    def __init__(self):
        self.fc1 = nn.Linear(784, 450, need_bias=True)
        self.fc2 = nn.Linear(450, 250, need_bias=True)
        self.fc3 = nn.Linear(250, 50, need_bias=True)
        self.fc4 = nn.Linear(50, 10, need_bias=True)

    def forward(self, x: Tensor) -> Tensor:
        "TODO: define forward pass"
        x = Tensor(x.data.reshape(-1, 28*28), requires_grad=x.requires_grad, depends_on=x.depends_on)
        x = activation.relu(self.fc1(x))
        x = activation.relu(self.fc2(x))
        x = activation.leaky_relu(self.fc3(x), alpha=0.075)
        x = activation.relu(self.fc4(x))
        return activation.softmax(x)

model = MyModel()
print(model.summary())

### Choose a Loss Function

In [8]:
"TODO: choose a proper loss function"
criterion = loss_func.CategoricalCrossEntropy()

### Choose an Optimizer

In [9]:
"TODO: choose a proper optimizer"
optimizer = optim.Adam()

### Training Model

In [ ]:
"TODO: define number of epoch and train batches of data on your model. also test each epoch."
EPOCH = 30
optimizer = optim.Adam(model.parameters())

for epoch in range(EPOCH):
    train_acc = []
    test_acc = []
    best_train_acc = 0.0
    best_test_acc = 0.0

    "TODO: train over your defined batches and save train accuracy for each epoch."
    
    "TODO: test your model after each training and save test accuracy for each epoch."
    
  
    for images, labels in data_loader.getTrain():
        outputs = model(images)

        max_value = 9
        one_hot_matrix = np.eye(10)
        one_hot_encoded = one_hot_matrix[labels.data]
        loss = loss_func.CategoricalCrossEntropy(outputs, Tensor(one_hot_encoded, requires_grad=labels.requires_grad, depends_on=labels.depends_on))
        
        optimizer.zero_grad()
        loss.backward(grad=Tensor(0.0))
        optimizer.step()

        predicted = np.argmax(outputs, axis=1)
        train_count = labels.size(0)
        train_acc.append((predicted == labels).sum().item() / train_count)
        
        if train_acc[-1] > best_train_acc:
            best_train_acc = train_acc[-1]
    
    print(f'Epoch [{epoch+1}/{EPOCH}], Train Accuracy: {train_acc[-1]:.4f}')
    
    correct_test = 0
    total_test = 0
    
    for images, labels in data_loader.getTest():
        outputs = model(images)
        predicted = np.argmax(outputs, axis=1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

    test_accuracy = correct_test / total_test

    if best_test_acc < test_accuracy:
        best_test_acc = test_accuracy
    
    test_acc.append(test_accuracy)

    print(f'Test Accuracy: {test_accuracy:.4f}')

### Evaluate Model Performance

In [ ]:
plt.plot(train_acc, label='train accuracy')
plt.plot(test_acc, label = 'test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show();

print("\nOn train - best accuracy: {:.2f}, final accuracy: {:.2f}".format(best_train_acc, train_acc[-1]))
print("On test - best accuracy: {:.2f}, final accuracy: {:.2f}".format(best_test_acc, test_acc[-1]))